In [1]:
import pickle
import torch
from torch import nn
import util
import model
import dataset

In [2]:
with open('sec-bert.pkl', 'rb') as file:
    net = pickle.load(file)

In [3]:
batch_size, max_len = 512, 64
train_iter, vocab = util.load_csic_data(batch_size, max_len)

In [4]:
devices = util.try_all_gpus()

In [8]:
def get_bert_encoding(net, tokens_a, tokens_b=None):
    tokens, segments = dataset.get_tokens_and_segments(tokens_a, tokens_b)
    print(f'debug tokens: {tokens}')
    token_ids = torch.tensor(vocab[tokens], device=devices[0]).unsqueeze(0)
    segments = torch.tensor(segments, device=devices[0]).unsqueeze(0)
    valid_len = torch.tensor(len(tokens), device=devices[0]).unsqueeze(0)
    encoded_X, _, _ = net(token_ids, segments, valid_len)
    return encoded_X

In [9]:
tokens_a = ['id=3', 'nombre=vino+rioja', 'precio=100', 'cantidad=55', 'b1=a%f1adir+al+carrito']
encoded_text = get_bert_encoding(net, tokens_a)
encoded_text_cls = encoded_text[:, 0, :]
encoded_text_crane = encoded_text[:, 2, :]
encoded_text.shape, encoded_text_cls.shape, encoded_text_crane[0][:3]

debug tokens: ['<cls>', 'id=3', 'nombre=vino+rioja', 'precio=100', 'cantidad=55', 'b1=a%f1adir+al+carrito', '<sep>']


(torch.Size([1, 7, 128]),
 torch.Size([1, 128]),
 tensor([-0.7415, -0.7096, -0.5571], device='cuda:0', grad_fn=<SliceBackward0>))

In [7]:
encoded_text

tensor([[[ 2.4318e-02, -6.0776e-01, -6.1173e-03, -4.7865e-01, -3.2036e-01,
           1.7295e+00, -4.4568e-01, -4.4152e-02,  1.4999e+00, -1.8206e+00,
          -2.1194e-01, -5.7554e-01,  8.1707e-02,  6.9063e-02,  1.0705e+00,
           1.1003e+00,  1.5845e-01, -1.5352e-01,  8.6553e-01, -6.2746e-02,
          -4.5392e-01, -8.3502e-02,  1.4104e+00, -2.0147e+00, -1.6687e-01,
          -3.4086e-02,  9.5495e-01, -4.4713e-02,  2.9681e-02,  1.0971e-01,
          -1.3834e+00, -3.8244e-01,  1.4753e+00, -7.7375e-02,  1.5050e-01,
           1.0939e-01, -1.8879e-02, -1.8138e-01, -1.3491e+00,  1.8392e+00,
          -1.6772e+00, -9.7508e-01,  2.2323e+00,  1.3176e+00, -8.0969e-01,
           1.8353e+00, -9.9705e-01, -4.2779e-01, -9.5369e-02, -6.1311e-01,
          -1.4419e-01,  1.9899e-01,  1.3087e+00,  7.3869e-01, -2.1083e+00,
           2.3978e+00, -1.5885e+00,  1.3774e+00, -1.7780e-01, -1.7160e+00,
           1.3866e+00, -1.1748e+00,  5.0195e-02,  2.8820e-02, -1.8860e+00,
          -1.1015e+00, -1